<a href="https://colab.research.google.com/github/SY-256/llms-from-scratch/blob/main/notebooks/ch02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter2: Working with Text Data

## 2.2 テキストをトークン化する

In [ ]:
from importlib.metadata import version

print("torch version: ", version("torch"))
print("tiktoken version: ", version("tiktoken"))

In [ ]:
import os
import requests

if not os.path.exists("the-verdict.txt"):
    url = (
        "https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt"
    )
    file_path = "the-verdict.txt"

    response = requests.get(url, timeout=30)
    response.raise_for_status()
    with open(file_path, "wb") as f:
        f.write(response.content)


In [ ]:
with open("the-verdict.txt", "r") as f:
    raw_text = f.read()

print("Total number of character. ", len(raw_text))
print(raw_text[:99])

In [ ]:
# 空白で文字列分割
import re
text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

- テキストをすべて小文字にするのはやめる -> 大文字はLLMが固有名詞と普通名詞を区別し、文の構造を理解し、大文字を正しく使ったテキストの生成を学習するのに役立つため安易に小文字に変換しない

In [ ]:
# ホワイトスペース+コンマ、ピリオドでの正規表現分割
result = re.split(r'([,.]|\s)', text)
print(result)

In [ ]:
# ホワイトスペース不要なので除く
item = re.split(r'([,.]|\s)', text)
result = [item for item in result if item.strip()]
print(result)

- ホワイトスペースを取り除くか／取り除かないかはアプリケーションとその要件によって変わってくので、一概にすべて取り除けばよいという訳でもない
- テキストの正確な構造に敏感なモデル（Pythonコード生成）であればホワイトスペースは残すべき

In [ ]:
# 疑問符、引用符、ダッシュなどの他の種類の句読点や特殊文字も扱えるようにする
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item for item in result if item.strip()]
print(result)

In [ ]:
# "the-verdict.txt"に適用
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item for item in preprocessed if item.strip()]
print(preprocessed[:30])

In [ ]:
# 合計のトークン数
print(len(preprocessed))

## 2.3 トークンをトークンIDに変換する

In [ ]:
# 一意なトークンから成るリストを作成し、アルファベット順に並び替える
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

In [ ]:
# 語彙を作成して、最初の51個のエントリを出力
vocab = {token:integer for integer, token in enumerate(all_words)}

In [ ]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

In [ ]:
# シンプルなテキストトークナイザーの実装
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab # encodeメソッドとdecodeメソッドでアクセスできるように語彙をクラス属性に格納
        self.int_to_str = {i:s for s,i in vocab.items()} # トークンIDからテキストトークンにマッピングするための逆引き辞書

    def encode(self, text):
        """入力テキストをトークンIDに変換"""
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

        preprocessed = [
            item for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        """トークンを変換してテキストに戻す"""
        text = " ".join([self.int_to_str[i] for i in ids])
        # 指定された句読点の前にあるスペースを削除
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [ ]:
# "The Verdict.txt"の内容を語彙にテキストをトークン化
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

In [ ]:
# トークンIDからテキストに戻せるか
print(tokenizer.decode(ids))

In [ ]:
# 訓練セット（語彙）に含まれていないテキストでもできるか
text = "Hello, do you like tea?"
print(tokenizer.encode(text))

- 語彙に含まれていないのでKeyErrorになる
- 語彙を増やすために大規模で多様な訓練データセットを考慮する必要がある

## 2.4 特別なコンテキストトークンを追加する
- 未知の単語に対処するためにトークナイザーを修正する必要がある
- 特別なコンテキストトークンの使い方と、そうしたトークンを追加することについても検討が必要
- コンテキストトークンを追加すると、テキストのコンテキストやその他の関連情報に対するモデル理解を向上させることができる
- 特別なトークンには、未知の単語や文章の境界を示すマーカーを含めることができる

### 特殊トークン
- 一部のトークナイザーは、LLMに追加の文脈を提供するために特殊トークンを使用する
- これらの特殊トークンには以下のようなものがある
- `[BOS]`（シーケンスの開始）はテキストの開始点を示す
- `[EOS]`（シーケンスの終了）はテキストの終了点を示す（通常、複数の無関係なテキスト（例. 異なるウィキペディア記事や異なる書籍など）を連結するために使用される）
- `[PAD]`（パディング）:バッチサイズが1より大きいLLMを訓練する場合に使用（長さの異なる複数のテキストを含める場合、パディングトークンで短いテキストを最長テキストの長さに調整し、全テキストの長さを均一化する）
- `[UNK]`:語彙に含まれない単語を表す

- GPT-2は上記のトークンは一切使用せず、複雑さを軽減するために`<|endpftext|>`トークンのみを使用する
- `<|endoftext|>`は前述の`[EOS]`トークンに相当する
- GPTもパディングに`<|endoftext|>`を使用（バッチ入力で訓練する際は通常マスクを使用し、パディングトークンには注意が向けないため、これらのトークンに内容に問題にならない）
- GPT-2は語彙以外に`<UNK>`トークンを使用せず、代わりにバイトペアエンコーディング（BPE）トークナイザーを採用。これは単語をサブワード単位に分解するもの

In [ ]:
# <|unk|>と<|endoftext|>の2つの特殊トークンを単語リストに追加
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer, token in enumerate(all_tokens)}

In [ ]:
# 元の語彙：1,130
len(vocab.items())

In [ ]:
# 更新された語彙の最後の5つ
# 追加した2つの特殊トークンが含まれていること確認
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

In [ ]:
# 未知の単語に対処するシンプルなテキストトークナイザー
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ] # 語彙になけれな"<|unk|>"で埋める

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # 指定された句読点の前にあるスペースを置き換える
        text = re.sub(r'\sL([,.:;?!()\'])', r'\1', text)
        return text

In [ ]:
# 試してみる
tokenizer = SimpleTokenizerV2(vocab)
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

In [ ]:
# トークナイズ&エンコードしてみる
tokenizer.encode(text)

In [ ]:
# デコード
tokenizer.decode(tokenizer.encode(text))

## 2.5 バイトペアエンコーディング（BPE）
- GPT-2はトークナイザーとしてBytePairエンコーディング（BPE）を採用
- これによりモデルは、事前定義された語彙にない単語を小さなサブワード単位や個々の文字に分解でき、語彙以外の単語を処理できるようになる
- 例えば、GPT-2の語彙に「unfamiliarword」という単語が存在しない場合、学習済みのBPEマージ設定に応じて、これを[「unfam」, [iliar], [word]]などのサブワード分解でトークン化することがある（サブワードが語彙にある前提）

In [ ]:
# BPEトークナイザーのインスタンス化
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
# 使い方
# トークナイズ&エンコーディング
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunknownPlace."
)
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

In [ ]:
# デコード
strings = tokenizer.decode(integers)
print(strings)

- BPEトークナイザーを使用することで未知の単語をサブワードや個々の文字に分割する
- 結果、どのような単語が来ても解析できるため、BPEトークナイザーでは、未知の単語を`<|unk|>`のような特殊トークンで置き換える必要がない -> トークンID振れさえすれば良い -> エンコード ⇔ デコードで対応できれば良い
- あらゆるテキストが処理できる

In [ ]:
## 練習問題 2-1
text = "Akwirw ier"
ids = tokenizer.encode(text)
print(f"トークンID: {ids}")
text_decode = tokenizer.decode(ids)
print(f"デコードテキスト: {text_decode}")

## 2.6スライディングウィンドウによるデータサンプリング
- 入力変数と目的変数のペアを生成する
- LLMの事前学習はテキストにおいて次に来る単語を予測する方法で行われる

In [ ]:
# BPEトークナイザー使って「The Veedict」をトークン化
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

In [ ]:
# 最初の50ワードを削除
enc_sample = enc_text[50:]

In [ ]:
# 次単語予測のため入力変数と目的変数のペアを作成
# xには入力トークン、yには（xを1つ後ろにシフトさせた）ターゲットトークン
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

In [ ]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

In [ ]:
# トークンIDをテキストに変換したバージョン
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

In [ ]:
# バッチ入力と目的変数のためのデータセット
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # トークナイザーで全テキストをトークン化
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max length+1"

        # スライディングウィンドウを使ってmax_lengthの長さのシーケンスに分割
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
# 入力変数と目的変数のペアでバッチを生成するデータローダー
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                      stride=128, shuffle=True, drop_last=True,
                      num_workers=0):
    # トークナイザーを初期化
    tokenizer = tiktoken.get_encoding("gpt2")

    # データセット作成
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last, # drop_last=Trueは、指定されたbatch_sizeよりも最後のバッチが短い場合に、訓練中の損失値のスパイクを防ぐためにそのバッチを除外
        num_workers=num_workers # 前処理に使用するCPU数
    )

    return dataloader

In [ ]:
with open("the-verdict.txt", "r") as f:
    raw_text = f.read()

In [ ]:
# コンテキストサイズが4のLLMで、バッチサイズが1のデータローダーをテスト
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

In [ ]:
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

In [ ]:
secod_batch = next(data_iter)
print(secod_batch)

In [ ]:
# 練習問題 2-2
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=8, stride=2, shuffle=False
)

In [ ]:
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

バッチサイズが小さいと訓練時に必要なメモリが少なくなるものの、モデルの更新時にノイズが増える（勾配の計算やパラメータの更新が不安定になる）

バッチサイズはトレードオフのハイパーパラメータ

In [ ]:
# データローダーで1よりも大きいバッチサイズでサンプリングを行う
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=4, stride=4, shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

1つのtorch.Tensorにbatch_size分
1つの要素はmax_length分

オーバーラップが多いと（モデルが同じデータを何度も見てしまい）過剰適合のリスクが高まる

## 2.7 トークン埋め込みを作成する
- トークンIDを埋め込みベクトルに変換する
- 下準備として、埋め込み層の重みをランダムな値で初期化する必要がある
- テキストをトークン化 -> テキストトークンをトークンIDに変換 -> トークンIDを埋め込みベクトルに変換する

### 連続値のベクトル表現（埋め込み）が必要なのは、GPT型のLLMが誤差逆伝播（バックプロパゲーションアルゴリズム）で訓練されたディープニューラルネットワークだから

In [ ]:
# トークンIDが埋め込みベクトルに変換される仕組みの確認
input_ids = torch.tensor([2, 3, 5, 1])

In [ ]:
# 6語の小さいな語彙
# サイズ3の埋め込みを作成（GPT-3の埋め込みサイズは12,288次元）
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [ ]:
# 埋め込み層の行列
print(embedding_layer.weight)

埋め込み層の重み行列には、小さな乱数値が含まれる。これらの値は,、LLM自体の最適化の一部として、LLMの訓練時に最適化される。

語彙に含まれるトークン（6つ）ごとに1つの行列があり、埋め込みベクトルの次元（３）ごとに1つの列がある

In [ ]:
# 埋め込み層をトークンIDに適用して埋め込みベクトルを作成
print(embedding_layer(torch.tensor([3])))

埋め込み層の重み行列のインデックス4に相当する値が取得できる

埋め込み層は事実上、トークンIDに基づいて埋め込み層の重い行列から行を取り出している（ルックアップ演算）

In [ ]:
print(embedding_layer(input_ids))

2.8 単語の位置をエンコードする
- トークン埋め込みは、基本的にはLLMの入力に適している
- LLMの欠点は、そのSelf-Attentionメカニズムにシーケンス内のトークンの位置や順序という概念がないこと
- トークンIDがシーケンス内のどの位置にあろうと、同じトークンIDを常に同じベクトル表現にマッピングする
- `相対位置埋め込み`と`絶対位置埋め込み`で位置を認識できる仕組みを与えることでトークン間の順序や関係を理解するLLMの能力を向上させることができる

In [ ]:
vocab_size = 50257 #（BPEトークナイザーの語彙数）
output_dim = 256

toke_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [ ]:
# データローダーをインスタンス化
max_length = 4

dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [ ]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

トークンIDのテンソルは8 x 4次元で、データバッチがそれぞれ4つのトークンを持つ8つのテキストサンプルで構成されている

In [ ]:
# トークンIDを256次元のベクトルに埋め込む
token_embeddings = toke_embedding_layer(inputs)
print(token_embeddings)
print(token_embeddings.shape)

GPTモデルが使っている絶対位置埋め込みの場合は、token_embedding_layerと同じ埋め込み次元を持つ別の埋め込み層を作成すれば良い

In [ ]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [ ]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings)
print(pos_embeddings.shape)

In [ ]:
# 位置埋め込みテンソルは直接トークン埋め込みに追加できる（次元数が一致するから）
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

In [ ]:
torch.arange(max_length)

In [ ]:
pos_embedding_layer.weight

In [ ]:
pos_embedding_layer